## Data Pipeline steps
1. Data Extraction
2. Quality Checks/Assessment
3. Data Cleaning/Transformation
4. Data Validation
5. Feature Engineering
6. Data Loading


In [ ]:
!pip install pandas

In [ ]:
#import nececessary libraries
import pandas as pd

## Step1 - Data Extraction

In [ ]:
dtype_dict = {'phone_number' : str,
              'customer_id': str}

In [ ]:
df_cdr =pd.read_csv('teleconnect_cdr_data - teleconnect_cdr_data.csv',dtype=dtype_dict)

In [ ]:
df_cdr.head()

In [ ]:
df_cdr.columns

In [ ]:
df_cdr.shape

In [ ]:
df_cdr.info()

## Step 2 - Data Quality Assessment/Checks

### Check for missing values

In [ ]:
df_cdr.isnull().sum()

In [ ]:
# checking for duplicate
df_cdr.duplicated().sum()

In [55]:
# check for inconsistent values
df_cdr['call_type'].value_counts()

call_type
sms      894
voice    866
data     846
Data     845
SMS      832
Voice    829
Name: count, dtype: int64

In [ ]:
df_cdr['call_success'].value_counts()

In [ ]:
df_cdr['network_type'].value_counts()

In [ ]:
df_cdr['roaming'].value_counts()

In [ ]:
df_cdr['phone_number'].str.len().value_counts()

### Step 3 - Data Cleaning/Transformation

In [ ]:
df_clean = df_cdr.copy()

In [ ]:
# Handle missing values
df_clean = df_clean.dropna(subset=['customer_id','phone_number'])

In [ ]:
df_clean.isnull().sum()

In [ ]:
#fill missing values
fill_values ={
    'call_duration_seconds': 0,
    'data_usage_mb':0,
    'signal_strength_dbm':df_clean['signal_strength_dbm'].median(),
    'tower_id':'UNKNOWN'
}
df_clean = df_clean.fillna(value=fill_values)

In [ ]:
df_clean =df_clean.drop_duplicates()

In [ ]:
df_clean.duplicated().sum()

### Standardize text formarts

In [ ]:
df_clean['call_type'] = df_clean['call_type'].str.lower().str.title()
df_clean['call_type'].value_counts()

In [ ]:
# standadize network type
df_clean['network_type'] = df_clean['network_type'].str.upper()
df_clean['network_type'].value_counts()

In [ ]:
# create function to standardize categorical columns
def standardize_boolean(value):
    if pd.isna(value):
        return None
    value_str = str(value).lower()
    if value_str in ['yes','true','1','y']:
        return True
    elif value_str in ['no','false','0','n']:
        return False
    else:
        return None
    

In [ ]:
df_clean['call_success'] = df_clean['call_success'].apply(standardize_boolean)
df_clean['roaming'] = df_clean['roaming'].apply(standardize_boolean)

In [ ]:
df_clean['call_success'].value_counts()

In [ ]:
df_clean['roaming'].value_counts()

In [ ]:
# convert timestamp to datetime
df_clean['call_timestamp'] = pd.to_datetime(df_clean['call_timestamp'])

In [ ]:
df_clean.info()

### Step 4 - Data Validation

In [ ]:
# create function to validate phone numbers
def validate_phone_number(phone):
    if pd.isna(phone):
        return False
    phone_str = str(phone)
    return len(phone_str) == 11

In [ ]:
df_clean['phone_valid'] = df_clean['phone_number'].apply(validate_phone_number)

In [ ]:
# noticed that zero was removed from the last digit of the phone number
# add back the zeros
df_clean['phone_number'] = df_clean['phone_number'].astype(str).str.ljust(11, '0')


In [ ]:
# removing +234 and replacing them with 0
df_clean['phone_number'] = (
    df_clean['phone_number']
    .astype(str)
    .str.replace(r'^(\+234|234)', '0', regex=True)
)


### signal strength validation
### -120 and -30

In [ ]:
#signal strength range
df_clean['signal_valid'] = df_clean['signal_strength_dbm'].between(-120,-30)






In [ ]:
df_clean['signal_valid'].value_counts()

### Feature Engineering

In [ ]:
# time base features
df_clean['call_hour'] = df_clean['call_timestamp'].dt.hour
df_clean['day_of_week'] = df_clean['call_timestamp'].dt.day_name()
df_clean['call_month'] = df_clean['call_timestamp'].dt.month_name()

In [ ]:
df_clean.head()

In [ ]:
df_clean['phone_valid'] = df_clean['phone_number'].str.ljust(11, '0')
df_clean['phone_valid'].value_counts()


In [ ]:
df_clean['phone_valid'].value_counts()

In [ ]:
df_clean[df_clean['phone_number'].astype(str).str.len() < 11]


In [ ]:
df_clean['phone_valid'] = df_clean['phone_number'].str.ljust(11, '0')
df_clean['phone_valid'].value_counts()

In [ ]:
# segmenting customers according to their revenue
def segment_customer(amount):
    if amount <= 333:
        return 'Low'
    elif amount <=666:
        return 'Medium'
    else:
        return'High'
df_clean['customer_segment'] = df_clean['revenue_naira'].apply(segment_customer)

In [ ]:
df_clean.head()

In [ ]:
#create a new column to categorize signal strength
def categorize_signal(signal):
    if pd.isna(signal):
        return 'Unknown'
    if signal >= -50:
        return 'Excellent'
    elif signal >= -75:
        return 'Good'
    elif signal >= -90:
        return 'Fair'
    else:
        return'Poor'
df_clean['signal_quality'] = df_clean['signal_strength_dbm'].apply(categorize_signal)

## Step 6 - Data Loading

In [ ]:
df_clean.to_csv('cdr_cleaned.csv',index=False)

In [ ]:
# ============================
# FULL POSTGRES CONNECTION SETUP
# ============================

# Import necessary libraries
import psycopg2
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load the .env file
# If your .env is NOT in the same folder as your script,
# change this to: load_dotenv("../.env") or load_dotenv("../../.env")
load_dotenv()

# Read environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

# Sanity check
print("HOST:", DB_HOST)
print("PORT:", DB_PORT)
print("USER:", DB_USER)
print("PASSWORD:", DB_PASSWORD)
print("DB NAME:", DB_NAME)

# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

print("Engine created successfully!")


In [ ]:
df_clean.to_sql('teleconnect_cdr', engine, if_exists='replace', index=False)
print('Data loaded to database successfully')
